# Kafka Producer Test
1. Read this CSV with headers using spark.
2. Publish these records into Kafka in streaming fashion.

### Test dummy (practice)

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, to_json, struct

In [2]:
spark = SparkSession.builder \
    .appName("SimpleKafkaProducerTest") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.5") \
    .getOrCreate()

In [3]:
spark.sparkContext.setLogLevel("WARN")

In [4]:
rate_df = spark.readStream \
    .format("rate") \
    .option("rowsPerSecond", 1) \
    .load()

In [5]:
test_df = rate_df.select(
    to_json(
        struct(
            lit("Test Message").alias("message"),
            rate_df["value"].alias("id")
        )
    ).alias("value")
)

In [6]:
kafka_bootstrap_servers = "kafka:9092"  
topic = "test-topic"

In [7]:
query = test_df.writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("topic", topic) \
    .option("checkpointLocation", "/tmp/kafka_test_checkpoint") \
    .outputMode("append") \
    .start()

In [9]:
# print("Simple test streaming producer is running. Check your Kafka consumer for messages.")
# query.awaitTermination()

---

# Kafka_Producer Code

In [2]:
from pyspark.sql import SparkSession as ss, Row as r
from pyspark.sql.functions import to_json, struct, col, lit
from pyspark.sql.types import LongType as lt

In [3]:
sprk = ss.builder \
    .appName("KafkaProducer") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.5") \
    .config("spark.jars.repositories", "https://repos.spark-packages.org") \
    .getOrCreate()

In [4]:
sprk.sparkContext.setLogLevel("WARN")

In [5]:
fp = "/data/dataset.csv"
df = sprk.read.csv(fp, header=True, inferSchema=True).cache()

In [8]:
print("Preview of static CSV DataFrame:")
df.show(5)

Preview of static CSV DataFrame:
+----------------+-------------------+----------------+-----------------------------+------------------+
|       Date/Time|LV ActivePower (kW)|Wind Speed (m/s)|Theoretical_Power_Curve (KWh)|Wind Direction (°)|
+----------------+-------------------+----------------+-----------------------------+------------------+
|01 01 2018 00:00|   380.047790527343|5.31133604049682|             416.328907824861|  259.994903564453|
|01 01 2018 00:10|    453.76919555664|5.67216682434082|             519.917511061494|   268.64111328125|
|01 01 2018 00:20|   306.376586914062|5.21603679656982|             390.900015810951|  272.564788818359|
|01 01 2018 00:30|   419.645904541015|5.65967416763305|             516.127568975674|  271.258087158203|
|01 01 2018 00:40|   380.650695800781|5.57794094085693|             491.702971953588|  265.674285888671|
+----------------+-------------------+----------------+-----------------------------+------------------+
only showing top 5 row

In [18]:
def add_idx_row(row, index):
    row_dict = row.asDict()
    row_dict["row_id"] = index
    return r(**row_dict)

In [19]:
rdd_w_idx = df.rdd.zipWithIndex().map(lambda x: add_idx_row(x[0], x[1]))
new_schema = df.schema.add("row_id", lt())
df_w_idx = sprk.createDataFrame(rdd_w_idx, schema=new_schema)

In [20]:
df_w_idx.printSchema()

root
 |-- Date/Time: string (nullable = true)
 |-- LV ActivePower (kW): double (nullable = true)
 |-- Wind Speed (m/s): double (nullable = true)
 |-- Theoretical_Power_Curve (KWh): double (nullable = true)
 |-- Wind Direction (°): double (nullable = true)
 |-- row_id: long (nullable = true)
 |-- row_id: long (nullable = true)



In [21]:
strmng_df = sprk.readStream \
    .format("rate") \
    .option("rowsPerSecond", 10) \
    .load()

In [22]:
strmng_df = strmng_df.withColumnRenamed("value", "row_id")

In [23]:
print(df_w_idx.count())

Py4JJavaError: An error occurred while calling o90.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 4 times, most recent failure: Lost task 0.3 in stage 10.0 (TID 11) (172.18.0.5 executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^
  File "/opt/bitnami/spark/python/pyspark/sql/session.py", line 1459, in prepare
    v

In [24]:
strmng_df = strmng_df.filter(col("row_id") < lit(df_w_idx.count()))

Py4JJavaError: An error occurred while calling o90.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 4 times, most recent failure: Lost task 0.3 in stage 11.0 (TID 15) (172.18.0.5 executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^
  File "/opt/bitnami/spark/python/pyspark/sql/session.py", line 1459, in prepare
    v

In [ ]:
new_df = strmng_df.join(df_w_idx, on="row_id", how="left_outer").drop("timestamp")

In [25]:
new_df.printSchema()

root
 |-- row_id: long (nullable = true)
 |-- Date/Time: string (nullable = true)
 |-- LV ActivePower (kW): double (nullable = true)
 |-- Wind Speed (m/s): double (nullable = true)
 |-- Theoretical_Power_Curve (KWh): double (nullable = true)
 |-- Wind Direction (°): double (nullable = true)



In [26]:
out = new_df.select(
    to_json(struct([col(x) for x in new_df.columns])).alias("value")
)

In [27]:
out.printSchema()

root
 |-- value: string (nullable = true)



In [28]:
query = out.writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("topic", "xenon-topic") \
    .option("checkpointLocation", "/tmp/kafka_checkpoint") \
    .outputMode("append") \
    .trigger(once=True) \
    .start()

query.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/bitnami/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/bitnami/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/socket.py", line 720, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [29]:
sprk.stop()